# Run NMRfilter

Navegate to the NMRfilter directory to proceed with instalations according to https://github.com/stefhk3/nmrfilter
- The program is stored in a directory named nmrfilter
- Projects are stored in different directories [projectFOLDER] within __a main directory__ (e.g. nmrfilterprojects) in the same level to nmrfilter. 
    - This path should be set at the __nmrproc.properties file__ together with input parameters.
- Once this is set, you can run the following code, chosing your project from the dropdown.

In [3]:
import configparser
import configparser
import os
import subprocess
import sys
from clustering import *
from clusterlouvain import *
from similarity import *
from util import *
import ipywidgets as widgets
projects = [f for f in os.listdir(datapath) if os.path.isdir(os.path.join(datapath, f))]
menu = widgets.Dropdown(
       options=projects,
       description='Chose your project:')
menu

Dropdown(description='Project:', options=('artificialwHMBC_DL', 'dadosMJCOLMARloose_DL', 'pbolduswithhsqctocsy…

In [7]:
project = menu.value
print(project)
cp = readprops(project)
datapath=cp.get('datadir')
print(datapath)
checkprojectdir(datapath, project, cp)
print(cp)

brugmansiaAlkaloids_DL
/home/shk3/git/nmrfilterprojects
{'datadir': '/home/shk3/git/nmrfilterprojects', 'msmsinput': 'testall.smi', 'predictionoutput': 'resultprediction.csv', 'result': 'result.txt', 'solvent': 'Methanol-D4 (CD3OD)', 'tolerancec': '0.2', 'toleranceh': '0.02', 'spectruminput': 'realspectrum.csv', 'clusteringoutput': 'cluster.txt', 'rberresolution': '.2', 'louvainoutput': 'clusterslouvain.txt', 'usehsqctocsy': 'false', 'usehmbc': 'true', 'dotwobonds': 'false', 'usedeeplearning': 'true', 'debug': 'true'}


In [19]:
! java -cp "./*" uk.ac.dmu.simulate.Convert $project
sdf=cp.get("msmsinput")[0:-4]+".sdf"
if cp.get('usedeeplearning'):
    %cd respredict
    ! python3 predict_standalone.py --filename $datapath$os.sep$project$os.sep$sdf --format sdf --nuc 13C --sanitize --addhs false > $datapath$os.sep$project{os.sep}predc.json
    ! python3 predict_standalone.py --filename $datapath$os.sep$project$os.sep$sdf --format sdf --nuc 1H --sanitize --addhs false > $datapath$os.sep$project{os.sep}predh.json
    %cd ..
! java -cp "./*" uk.ac.dmu.simulate.Simulate $project

true_testall.sdf_/home/shk3/git/nmrfilterprojects//opt/git/nmrfilter/respredict
/opt/git/nmrfilter
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscien

org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement

org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement

org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement

org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement

org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement not included in bounds calculation
org.openscience.cdk.renderer.elements.Bounds WARN: class org.openscience.cdk.renderer.elements.AtomSymbolElement

In [20]:
print("Clustering the peaks in the measured spectrum...")
cluster2dspectrum(cp, project)
print("Detecting communities in the measures spectrum...")
cluster2dspectrumlouvain(cp, project)
print("Calculating best hits in your compounds...")
similarity(cp, project)

Clustering the peaks in the measured spectrum...
Detecting communities in the measures spectrum...
Calculating best hits in your compounds...
1: CN1C2CC(CC1C1C2O1)OC(=O)C(CO)c1ccccc1/Hyoscine, distance: 0.03, standard deviation: 1.00, matching rate: 19/23, 82.61% (HMBC), matching rate: 11/11, 100.00% (HSQC)
2: CN1C2CCC1C(C(C2)OC(=O)C3=CC=CC=C3)C(=O)OC/Pseudococaine, distance: 0.27, standard deviation: 1.00, matching rate: 10/30, 33.33% (HMBC), matching rate: 9/12, 75.00% (HSQC)
3: CCOC(=O)C1C2CCC(N2C)CC1OC(=O)C3=CC=CC=C3/Homocaine, distance: 0.28, standard deviation: 1.00, matching rate: 10/30, 33.33% (HMBC), matching rate: 9/13, 69.23% (HSQC)
4: CN1C2CCC1C(C(C2)OC(=O)C3=CC=CC=C3)C(=O)O/Benzoylecogonine, distance: 0.33, standard deviation: 1.00, matching rate: 7/29, 24.14% (HMBC), matching rate: 7/11, 63.64% (HSQC)
5: CN1C2CC(CC1C(C2)O)OC(=O)C(CO)C3=CC=CC=C3/Racanisodamine, distance: 0.09, standard deviation: 0.72, matching rate: 18/28, 64.29% (HMBC), matching rate: 10/13, 76.92% (HSQC